In [ ]:
# create database Demo_table.db spaceconnect to existing database
import sqlite3
conn = sqlite3.connect('.\sql_db\Demo_table.db')
cur = conn.cursor()


In [ ]:
# create table in database
cur.execute('''CREATE TABLE CUSTOMER(
                User_ID INTEGER PRIMARY KEY NOT NULL,
                Product_ID INTEGER NOT NULL,
                Name TEXT NOT NULL,
                Gender TEXT NOT NULL,
                AGE INTEGER NOT NULL,
                CITY TEXT);
                ''')





In [ ]:
#we can execute SQL statements using the execute() method. So let’s insert a row of data!
cur.execute('''Insert Into Customer ('User_ID','Product_ID','Name','Gender','AGE','CITY') Values (1006, 3, 'Princess Diana', 'Female', 28, 'Amazons');''')


In [ ]:
#if we want to write multiple Insert commands in a single go? We could use the executescript() method instead:
#Execute multiple commands at once
cur.executescript('''Insert Into CUSTOMER Values
	                (1005, 3, 'Clark Kent', 'Male', 36, 'Metropolis');

	                Insert Into CUSTOMER Values
	                (1003, 4, 'Bruce Wayne', 'Male', 39, 'Gotham City');

	                ''')


In [ ]:
#use the executemany() method without having to repeatedly write the Insert Into command every time! executemany() actually executes an SQL command using an iterator to yield the values:
# Insert maultiple values into table at once
customers = [(1004, 2, 'John Wick', 'Male', 32, 'New York'),
	         (1001, 1, 'Tony Stark', 'Male', 35, 'New York'),
	         (1002, 3, 'Gordon Ramsey', 'Male', 38, 'London')
	            ]
cur.executemany('Insert Into CUSTOMER Values (?,?,?,?,?,?)', customers)


In [ ]:
#To fetch only a single record from the database, we can use the fetchone() method:
cur.execute('SELECT * FROM CUSTOMER;').fetchone()


(1001, 1, 'Tony Stark', 'Male', 35, 'New York')

In [ ]:
#To fetch multiple rows, you can execute a SELECT statement and iterate over it directly using only a single call on the Cursor object:
# iterate over the rows
for row in cur.execute('SELECT Name FROM CUSTOMER;'):
  print(row)


('Tony Stark',)
('Gordon Ramsey',)
('Bruce Wayne',)
('John Wick',)
('Clark Kent',)
('Princess Diana',)


In [ ]:
#a better way of retrieving multiple records would be to use the fetchall() method which returns all the records in a list format:
# Fetch all rows of query result which returns a list
cur.execute('SELECT * FROM CUSTOMER;').fetchall()



[(1001, 1, 'Tony Stark', 'Male', 35, 'New York'),
 (1002, 3, 'Gordon Ramsey', 'Male', 38, 'London'),
 (1003, 4, 'Bruce Wayne', 'Male', 39, 'Gotham City'),
 (1004, 2, 'John Wick', 'Male', 32, 'New York'),
 (1005, 3, 'Clark Kent', 'Male', 36, 'Metropolis'),
 (1006, 3, 'Princess Diana', 'Female', 28, 'Amazons')]

In [ ]:
#Pandas let us quickly write our data from a dataframe into a database using the to_sql() method.
#The method takes the table name and Connection object as its arguments.
#We use the dataframes from the Food Demand Forecasting hackathon on the DataHack platform which has three dataframes:
# order information, meal information, and center fulfillment information.
import pandas as pd
import sqlite3


# read csv files
df_train = pd.read_csv('/content/train1.csv')
df_meal = pd.read_csv('/content/meal_info.csv')
df_center = pd.read_csv('/content/fulfilment_center_info.csv')


In [ ]:
# connect to database
conn = sqlite3.connect('\sql_db\FOOD.db')
cur = conn.cursor()


# load dataframes into database
df_train.to_sql("train", conn)
df_meal.to_sql('meal',conn)
df_center.to_sql('centers',conn)


77

In [ ]:
# We can check the values in the tables using the read_sql_query() method:
df = pd.read_sql_query("select * from meal;", conn)
df.head()

,index,meal_id,category,cuisine
0,0,1885,Beverages,Thai
1,1,1993,Beverages,Thai
2,2,2539,Beverages,Thai
3,3,1248,Beverages,Indian
4,4,2631,Beverages,Indian


In [ ]:
# We can check the values in the tables using the real_sql_query() method:
df = pd.read_sql_query("select * from centers;", conn)
df.head()

,index,center_id,city_code,region_code,center_type,op_area
0,0,11,679,56,TYPE_A,3.7
1,1,13,590,56,TYPE_B,6.7
2,2,124,590,56,TYPE_C,4.0
3,3,66,648,34,TYPE_A,4.1
4,4,94,632,34,TYPE_C,3.6


In [ ]:
# We can check the values in the tables using the real_sql_query() method:
df = pd.read_sql_query("select * from train;", conn)
df.head()

,index,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1,1466964,1,55,1993,136.83,135.83,0,0,270
2,2,1346989,1,55,2539,134.86,135.86,0,0,189
3,3,1338232,1,55,2139,339.50,437.53,0,0,54
4,4,1448490,1,55,2631,243.50,242.50,0,0,40


In [ ]:
#We can use it to query and retrieve only the Indian cuisine meals from the meal table:
# WHERE clause
df = pd.read_sql_query('''Select * from meal
	                        Where cuisine='Indian'; ''', conn)
df

,index,meal_id,category,cuisine
0,3,1248,Beverages,Indian
1,4,2631,Beverages,Indian
2,16,1109,Rice Bowl,Indian
3,17,2290,Rice Bowl,Indian
4,26,2139,Beverages,Indian
5,33,2492,Desert,Indian
6,35,1727,Rice Bowl,Indian
7,36,1902,Biryani,Indian
8,37,1247,Biryani,Indian
9,38,2304,Desert,Indian


In [ ]:
#LIMIT clause returns only the specified number of records and is useful when there are a large number of records in the table.
# LIMIT clause
df = pd.read_sql_query('''Select * from meal
	                        Where category='Beverages'
	                        Limit 5;''', conn)
df


,index,meal_id,category,cuisine
0,0,1885,Beverages,Thai
1,1,1993,Beverages,Thai
2,2,2539,Beverages,Thai
3,3,1248,Beverages,Indian
4,4,2631,Beverages,Indian


In [ ]:
#We will group the records on the emailer_for_promotion column and apply the COUNT aggregate function on the id column since it contains unique values.
#This will return the total number of rows belonging to each group:
# GROUPBY statement
df = pd.read_sql_query('''Select Count(id) from train
	                        Group by emailer_for_promotion;''',conn)
df

,Count(id)
0,621
1,82


In [ ]:
	# ORDER BY statement
df = pd.read_sql_query('''Select count(id) as Email from train
	                        Group by emailer_for_promotion
	                        Order by Count(id);''',conn)
df


,Email
0,82
1,621


In [ ]:
# ORDER BY descending
df = pd.read_sql_query('''Select Count(center_type), center_type
	                        From centers
	                        Group By center_type
	                        Order by Count(center_type) Desc; ''',conn)
df

,Count(center_type),center_type
0,43,TYPE_A
1,19,TYPE_C
2,15,TYPE_B


In [ ]:
#HAVING is used to filter records after grouping. Hence, the HAVING clause is always used after the GROUP BY statement:
df = pd.read_sql_query('''Select Count(center_type), center_type
	                        From centers
	                        Group By center_type
	                        Having Count(center_type) > 15; ''', conn)
df.head()


,Count(center_type),center_type
0,43,TYPE_A
1,19,TYPE_C


In [ ]:
#In our database, we can retrieve data from the centers and train tables since they share the common attribute center_id:
# JOIN clause total orders per center_type
df = pd.read_sql_query('''Select Sum(train.num_orders) as "Total_Orders", centers.center_type from train
	                       Inner Join centers
	                       On train.center_id = centers.center_id
	                       Group By centers.center_type;''', conn)
df.head()

,Total_Orders,center_type
0,88858,TYPE_A
1,97135,TYPE_B
2,21818,TYPE_C


In [ ]:
#Let’s have a look at the table before the update:
df = pd.read_sql_query('''Select * from train
	                        Where emailer_for_promotion = 1; ''', conn)
df.head()

,index,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,14,1378227,1,55,1109,264.84,297.79,1,0,756
1,20,1044821,1,55,1971,259.99,320.13,1,1,798
2,24,1338119,1,55,1558,583.03,610.13,1,0,162
3,25,1188372,1,55,2581,583.03,612.13,1,0,312
4,26,1440008,1,55,1962,582.03,612.13,1,0,231


In [ ]:
#Decrease all the base prices by 10 for orders containing meals that had an email promotion:
# UPDATE statement
conn.execute('''Update train
	                Set base_price = base_price - 10
	                Where emailer_for_promotion = 1; ''')
#Finally, here’s a look at the updated table:
df = pd.read_sql_query('''Select * from train
	                        Where emailer_for_promotion = 1; ''', conn)
df.head()


,index,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,14,1378227,1,55,1109,264.84,287.79,1,0,756
1,20,1044821,1,55,1971,259.99,310.13,1,1,798
2,24,1338119,1,55,1558,583.03,600.13,1,0,162
3,25,1188372,1,55,2581,583.03,602.13,1,0,312
4,26,1440008,1,55,1962,582.03,602.13,1,0,231
